In [1]:
import pandas as pd
import numpy as np

# 1. Carga de datos (Asegúrense de subir el archivo BrFlights2.csv al entorno)
# Usamos low_memory=False porque el archivo parece grande
df = pd.read_csv('/content/BrFlights2.csv', encoding='latin1', low_memory=False)

print(f"Filas originales: {df.shape[0]}")

# 2. FILTRADO INICIAL
# Solo queremos vuelos que realmente sucedieron
df = df[df['Situacao.Voo'] == 'Realizado']

# Seleccionamos solo las columnas útiles para el MVP
cols_to_keep = [
    'Companhia.Aerea',
    'Aeroporto.Origem',
    'Aeroporto.Destino',
    'Partida.Prevista',
    'Partida.Real'
]
df = df[cols_to_keep].dropna()

# 3. INGENIERÍA DE VARIABLES (CREAR EL TARGET)
# Convertir strings a objetos datetime
df['Partida.Prevista'] = pd.to_datetime(df['Partida.Prevista'])
df['Partida.Real'] = pd.to_datetime(df['Partida.Real'])

# Calcular minutos de retraso (Real - Prevista)
# El resultado sale en segundos, dividimos por 60
df['delay_minutes'] = (df['Partida.Real'] - df['Partida.Prevista']).dt.total_seconds() / 60

# Definir TARGET: 1 si se atrasó más de 15 min, 0 si no.
df['target'] = np.where(df['delay_minutes'] > 15, 1, 0)

# 4. EXTRACCIÓN DE FECHAS (Para que el modelo entienda hora y día)
df['hora_partida'] = df['Partida.Prevista'].dt.hour
df['dia_semana'] = df['Partida.Prevista'].dt.dayofweek # 0=Lunes, 6=Domingo
df['mes'] = df['Partida.Prevista'].dt.month

# 5. RENOMBRAR PARA EL EQUIPO DE BACKEND (Opcional pero recomendado)
# Esto alinea el dataset con el JSON que te pidieron
df_final = df.rename(columns={
    'Companhia.Aerea': 'companhia',
    'Aeroporto.Origem': 'origem',
    'Aeroporto.Destino': 'destino'
})

# Ver cómo quedó
print("Distribución del Target (0=Pontual, 1=Atrasado):")
print(df_final['target'].value_counts(normalize=True))

print("\nPrimeras filas listas para entrenar:")
print(df_final[['companhia', 'origem', 'destino', 'hora_partida', 'target']].head())

# Guardar este dataset limpio para que todos trabajen sobre el mismo
df_final.to_csv('dataset_limpio_hackaton.csv', index=False)

Filas originales: 92793
Distribución del Target (0=Pontual, 1=Atrasado):
target
0    0.856434
1    0.143566
Name: proportion, dtype: float64

Primeras filas listas para entrenar:
               companhia         origem        destino  hora_partida  target
0  AMERICAN AIRLINES INC    Afonso Pena  Salgado Filho             8       0
1  AMERICAN AIRLINES INC  Salgado Filho          Miami            12       0
2  AMERICAN AIRLINES INC  Salgado Filho          Miami            12       0
3  AMERICAN AIRLINES INC  Salgado Filho          Miami            12       0
4  AMERICAN AIRLINES INC  Salgado Filho          Miami            12       0


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import joblib

# 1. CODIFICACIÓN (Transformar texto a números)
# Guardaremos los encoders para usarlos luego en la API
le_companhia = LabelEncoder()
le_origem = LabelEncoder()
le_destino = LabelEncoder()

# Usamos .astype(str) para evitar errores si hay algún numero mezclado
df_final['companhia_encoded'] = le_companhia.fit_transform(df_final['companhia'].astype(str))
df_final['origem_encoded'] = le_origem.fit_transform(df_final['origem'].astype(str))
df_final['destino_encoded'] = le_destino.fit_transform(df_final['destino'].astype(str))

# Definimos las variables X (features) e y (target)
features = ['companhia_encoded', 'origem_encoded', 'destino_encoded', 'hora_partida', 'dia_semana', 'mes']
X = df_final[features]
y = df_final['target']

# 2. SPLIT DE DATOS (80% entrenar, 20% testear)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3. ENTRENAMIENTO
# class_weight='balanced' es CLAVE para corregir el 85/15 que viste
print("Entrenando modelo (esto puede tardar un poco)...")
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced', max_depth=15)
model.fit(X_train, y_train)

# 4. EVALUACIÓN
y_pred = model.predict(X_test)

print("\n--- REPORTE DE RESULTADOS ---")
print(classification_report(y_test, y_pred))

print("\n--- MATRIZ DE CONFUSIÓN ---")
# [Verdaderos Negativos  Falsos Positivos]
# [Falsos Negativos      Verdaderos Positivos]
print(confusion_matrix(y_test, y_pred))

# 5. GUARDAR TODO (SERIALIZACIÓN)
# Guardamos el modelo y los encoders en un solo archivo o varios
artifacts = {
    'model': model,
    'le_companhia': le_companhia,
    'le_origem': le_origem,
    'le_destino': le_destino
}

joblib.dump(artifacts, 'flight_model_v1.joblib')
print("\n¡Modelo y encoders guardados exitosamente en 'flight_model_v1.joblib'!")

Entrenando modelo (esto puede tardar un poco)...

--- REPORTE DE RESULTADOS ---
              precision    recall  f1-score   support

           0       0.89      0.80      0.84      4242
           1       0.22      0.34      0.27       670

    accuracy                           0.74      4912
   macro avg       0.55      0.57      0.55      4912
weighted avg       0.79      0.74      0.76      4912


--- MATRIZ DE CONFUSIÓN ---
[[3402  840]
 [ 439  231]]

¡Modelo y encoders guardados exitosamente en 'flight_model_v1.joblib'!
